In [ ]:
# Link: https://machinelearningmastery.com/upsampling-and-transpose-convolution-layers-for-generative-adversarial-networks/

### UpSampling2D Layer

In [4]:
from numpy import asarray
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import UpSampling2D

# define input data
X = asarray([[1, 2],
             [3, 4]])
# show input data for context
print(X)

# reshape input data into one sample a sample with a channel
X = X.reshape((1, 2, 2, 1))

# define model
model = Sequential()
model.add(UpSampling2D(input_shape=(2, 2, 1)))
# summarize the model
model.summary()

# use the model to make a prediction --> upsample a provided input image
yhat = model.predict(X) 
# output will have dimensions --> convert back to 2x2 array
yhat = yhat.reshape((4, 4))
# summarize output
print(yhat)

[[1 2]
 [3 4]]
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d_2 (UpSampling2 (None, 4, 4, 1)           0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
[[1. 1. 2. 2.]
 [1. 1. 2. 2.]
 [3. 3. 4. 4.]
 [3. 3. 4. 4.]]


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import Conv2D

# for a UpSampling2D layer to actually be useful in a GAN, it needs to be followed by a Conv2D layer that learns to interpret the doubled input and translates it into something meaningful
# define model --> want it to produce a 10x10 image and take a 100 element vector asi nput
model = Sequential()
# define input shape, output enough activations for for 128 5x5 image
model.add(Dense(128 * 5 * 5, input_dim=100))
# reshape vector of activations into 128 feature maps with 5x5
model.add(Reshape((5, 5, 128)))
# upsample the 5x5 feature maps into a 10x10 feature map
model.add(UpSampling2D())
# the upsampled feature maps can be interpreted and filled in with a Conv2D layer
model.add(Conv2D(1, (3,3), padding='same'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3200)              323200    
_________________________________________________________________
reshape (Reshape)            (None, 5, 5, 128)         0         
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 10, 10, 1)         1153      
Total params: 324,353
Trainable params: 324,353
Non-trainable params: 0
_________________________________________________________________


## How to use the Conv2DTranspose Layer

In [7]:
from numpy import asarray
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2DTranspose


#it both performs the upsample operation and interprets the input data to fill in detail while upsampling
# kind of like a combined UpSampling2D and Conv2D layer

# define input data
X = asarray([[1, 2], # contrived input image of 2x2 pixels
             [3, 4]])
# show input data for context
print(X)

# reshape the input data into a sample with a channel
X = X.reshape((1, 2, 2, 1))

# define model
model = Sequential()
# takes 2x2 grayscale images as input and outputs the result of the operation
# this layer also upsamples and performs a convolution
    # need to specify both # of filters and size of filters (like in Conv2D layers)
    # stride of (2,2) bc the upsampling is achieved by the stride behavior of the convolution on the input
        # spaces out the input and rows/columns of 0.0 values are inserted to achieve it\
    # 1 filter, 1x1 kernel, 2x2 stride --> 2x2 input image is upsampled to 4x4
model.add(Conv2DTranspose(1, (1,1), strides=(2,2), input_shape=(2, 2, 1)))
# summarize the model
model.summary()
# define weights that they do nothing
weights = [asarray([[[[1]]]]), asarray([0])]
# store the weights in the model
model.set_weights(weights)
# make a prediction with the model
yhat = model.predict(X)
# reshape output to remove channel to make printing easier
yhat = yhat.reshape((4, 4))
# summarize output
print(yhat)

[[1 2]
 [3 4]]
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose (Conv2DTran (None, 4, 4, 1)           2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________
[[1. 0. 2. 0.]
 [0. 0. 0. 0.]
 [3. 0. 4. 0.]
 [0. 0. 0. 0.]]


# Simple generator model with the Conv2DTranspose Layer

In [8]:
# example of using transpose conv in a simple generator model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Conv2D
# define model
model = Sequential()
# define input shape, output enough activations for for 128 5x5 image
model.add(Dense(128 * 5 * 5, input_dim=100))
# reshape vector of activations into 128 feature maps with 5x5
model.add(Reshape((5, 5, 128)))
# double input from 128 5x5 to 1 10x10 feature map
    # 1 filter, 3x3 kernel size for the filter, padding = same ensures the output dimensions are 10x10
model.add(Conv2DTranspose(1, (3,3), strides=(2,2), padding='same'))
# summarize model
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3200)              323200    
_________________________________________________________________
reshape_1 (Reshape)          (None, 5, 5, 128)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 10, 10, 1)         1153      
Total params: 324,353
Trainable params: 324,353
Non-trainable params: 0
_________________________________________________________________
